In [ ]:
!pip install simpletransformers
!pip install sentencepiece
!pip install simpletransformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import torch
from transformers import PegasusForConditionalGeneration
from transformers import PegasusTokenizer
model_name = 'tuner007/pegasus_paraphrase'
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'



In [ ]:
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(torch_device)

In [ ]:
def get_response(input_text,num_return_sequences,num_beams):
  batch = tokenizer([input_text],truncation=True,padding='longest',max_length=60, return_tensors="pt").to(torch_device)
  translated = model.generate(**batch,max_length=60,num_beams=num_beams, num_return_sequences=num_return_sequences, temperature=1.5)
  tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
  return tgt_text

In [ ]:
#def get_response(df):
  

In [ ]:
num_beams = 1
num_return_sequences = 1
#context = "The ultimate test of your knowledge is your capacity to convey it to another."
context = "\n\nThe bill was introduced in the US Senate on Wednesday, and the US House of Representatives got a vote on it on Thursday afternoon. The US President Donald Trump is expected to sign it. [continues...]"
get_response(context,num_return_sequences,num_beams)


['The bill was approved by the US House of Representatives on Thursday and is expected to be signed by the US President.']

In [ ]:
from google.colab import drive
import pandas as pd

In [ ]:
drive.mount('/content/drive')
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Fine-Tuning/Data/cs577/full.csv')
#test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Fine-Tuning/Data/cs577/paraphrased.csv')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
a_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Fine-Tuning/Data/cs577/test.csv')
b_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Fine-Tuning/Data/cs577/train.csv')


In [ ]:
#drive.mount('/content/drive')
ptest_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Fine-Tuning/Data/cs577/paraphrased_test.csv')
ptrain_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Fine-Tuning/Data/cs577/paraphrased_train.csv')
pfull_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Fine-Tuning/Data/cs577/paraphrased.csv')

In [ ]:
def get_hidden_states(input_text,num_return_sequences,num_beams):
  batch = tokenizer([input_text],truncation=True,padding='longest',max_length=60, return_tensors="pt").to(torch_device)
  translated = model.generate(**batch,max_length=60,num_beams=num_beams, num_return_sequences=num_return_sequences, temperature=1.5)
  #tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
  return translated

In [ ]:
num_beams = 1
num_return_sequences = 1
row = df.iloc[0]
input_text = row['Text']
#print(row['Text'].type)
get_hidden_states(row['Text'], num_return_sequences,num_beams)

tensor([[    0,   110, 69098,   146,   656,   118,   200,   122,  6140,   112,
          6568,  2775,   113,  4585, 44078,   107,     1]], device='cuda:0')

In [ ]:
batch = tokenizer([input_text],truncation=True,padding='longest',max_length=60, return_tensors="pt")["input_ids"].to(torch_device)
input_ids = tokenizer([input_text], return_tensors='pt')['input_ids'].to(torch_device)
#print(input_ids)
outputs = model(decoder_input_ids=input_ids, input_ids=input_ids, output_hidden_states=True, output_attentions=True, return_dict=True)
#print(outputs.keys())
print(len(outputs['encoder_hidden_states']))
#decoder_hidden_states = model(decoder_input_ids=input_ids, input_ids=input_ids, decoder_hidden_states=True)
#print(decoder_hidden_states)
#model(**batch)

17


In [ ]:
hidden_state_1 = outputs['encoder_hidden_states'][0]
hidden_state_2 = outputs['encoder_hidden_states'][1]
hidden_state_3 = outputs['encoder_hidden_states'][2]
hidden_state_4 = outputs['encoder_hidden_states'][3]
print(hidden_state_1)

tensor([[[-9.0761e+00,  1.5051e-02,  1.8805e+01,  ...,  9.6768e+00,
          -1.1028e+00, -1.5832e+01],
         [-1.2719e+01,  1.1861e+00, -1.0433e+01,  ...,  8.0127e-01,
           1.1446e+01, -1.7039e+01],
         [ 4.2427e+00,  3.1670e+00, -4.7095e+00,  ..., -2.4411e+01,
           2.9047e+00,  8.9822e+00],
         ...,
         [ 5.6438e+00,  1.8921e+00, -5.1129e+00,  ..., -1.3354e+01,
           1.7963e+01,  6.2575e+00],
         [-5.7622e+00,  2.9635e-01,  3.0068e+01,  ...,  1.8978e+01,
          -1.0643e+01, -7.8753e+00],
         [-1.9474e+01, -6.8590e-01,  5.5070e+01,  ..., -3.7550e+00,
           1.2482e+01, -2.5112e+00]]], device='cuda:0', grad_fn=<AddBackward0>)


In [ ]:
arr = hidden_state_1.cpu().detach().numpy()
arr[0].size
hidden_state_1.size()

torch.Size([1, 37, 1024])

In [ ]:
# squeeze hidden states to [1, 37]
# multiple by batch and average the values
# take distance between prob distributions
batch.size()

torch.Size([1, 37])

In [ ]:
torch.squeeze(hidden_state_1).size()

torch.Size([37, 1024])

In [ ]:
print(outputs.keys())

odict_keys(['logits', 'past_key_values', 'decoder_hidden_states', 'decoder_attentions', 'cross_attentions', 'encoder_last_hidden_state', 'encoder_hidden_states', 'encoder_attentions'])


In [ ]:
outputs["encoder_last_hidden_state"]

tensor([[[-0.2803,  0.0013, -0.0394,  ..., -0.1274, -0.0186, -0.1597],
         [ 0.0430, -0.0340, -0.0628,  ..., -0.0191,  0.1115, -0.0469],
         [-0.2939, -0.0565, -0.0238,  ...,  0.1170, -0.0710, -0.0403],
         ...,
         [ 0.0513, -0.0866,  0.0062,  ...,  0.0477, -0.0542,  0.1495],
         [-0.0704,  0.1189,  0.0557,  ...,  0.3864, -0.1074,  0.1651],
         [ 0.0465, -0.1088,  0.0330,  ...,  0.0698,  0.0154,  0.0090]]],
       device='cuda:0', grad_fn=<NativeLayerNormBackward0>)

In [ ]:
outputs["encoder_last_hidden_state"].size()

torch.Size([1, 37, 1024])

In [ ]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
# Optional model configuration
config = {"output_hidden_states": True}
class_model_args = ClassificationArgs(num_train_epochs=1, config=config)
# Create a ClassificationModel
class_model = ClassificationModel(
    'bert',
    'bert-base-cased',
    num_labels=4,
    args=class_model_args,
    use_cuda=False
)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [ ]:
# Preparing eval data
eval_data = [
    ["Aragorn was the heir of Elendil", 1],
    ["Sam was the heir of Isildur", 0],
    ["Merrry is stronger than Pippin", 2],
]
eval_df = pd.DataFrame(eval_data)
outputs = class_model.predict(["Aragorn was the heir of Elendil"])

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
len(outputs)

4

In [ ]:
outputs1 = class_model.predict(["Sam was the heir of Isildur"])

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
import numpy as np
last_hidden_state = outputs[3][11]
print(last_hidden_state.shape)
matrix = np.zeros((1,128,768))
print(matrix.shape)

(1, 128, 768)
(1, 128, 768)


In [ ]:
auth0_df = pfull_df[pfull_df["Author"] == 0]
auth1_df = pfull_df[pfull_df["Author"] == 1]
auth2_df = pfull_df[pfull_df["Author"] == 2]
auth3_df = pfull_df[pfull_df["Author"] == 3]

In [ ]:
auth0_df = auth0_df.head(10)
auth1_df = auth1_df.head(10)
auth2_df = auth2_df.head(10)
auth3_df = auth3_df.head(10)

In [ ]:
for row in auth0_df.iterrows():
  text = row[1]['Paraphrased Text']
  ls = []
  ls.append(text)
  outputs = class_model.predict(ls)
  last_hidden_state = outputs[3][11]
  matrix = np.add(matrix, last_hidden_state)

datapoints = len(auth0_df)
avg_hidden_layer = np.divide(matrix, datapoints)
print(avg_hidden_layer)



  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

[[[ 0.43222222 -0.03437842 -0.08040119 ... -0.19378082  0.20505025
    0.00116063]
  [ 0.41906696 -0.48405848  0.41524604 ... -0.10736665  0.01738073
    0.13731344]
  [ 0.21510758 -0.04740865  0.08526784 ...  0.19564464 -0.10114785
   -0.02264921]
  ...
  [ 0.19655161  0.07655858 -0.01327556 ...  0.01707247  0.08506553
    0.15837857]
  [ 0.13941419  0.06662923  0.00623538 ... -0.01396576  0.063829
    0.14528191]
  [ 0.14872777 -0.03968009 -0.01069054 ... -0.04993806  0.08566071
    0.07410876]]]


In [ ]:
for row in auth1_df.iterrows():
  text = row[1]['Paraphrased Text']
  ls = []
  ls.append(text)
  outputs = class_model.predict(ls)
  last_hidden_state = outputs[3][11]
  matrix = np.add(matrix, last_hidden_state)

datapoints = len(auth1_df)
avg_hidden_layer1 = np.divide(matrix, datapoints)
print(avg_hidden_layer1)

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

[[[ 0.63020899 -0.02585093 -0.23868459 ... -0.29306221  0.52135603
    0.09684761]
  [ 0.67036926 -0.83553344  0.78067417 ... -0.12068925  0.09094616
    0.25863009]
  [ 0.21003958  0.05477012  0.08799138 ...  0.47297454  0.07909128
    0.0674088 ]
  ...
  [ 0.11341156  0.11639939 -0.02750509 ...  0.09065197  0.12598198
    0.30479325]
  [ 0.08778123  0.07327516 -0.01473755 ...  0.10070225  0.08853811
    0.22150477]
  [ 0.05107121 -0.03596337 -0.08985702 ...  0.06279567  0.12884143
    0.20988086]]]


In [ ]:
for row in auth2_df.iterrows():
  text = row[1]['Paraphrased Text']
  ls = []
  ls.append(text)
  outputs = class_model.predict(ls)
  last_hidden_state = outputs[3][11]
  matrix = np.add(matrix, last_hidden_state)

datapoints = len(auth2_df)
avg_hidden_layer2 = np.divide(matrix, datapoints)
print(avg_hidden_layer2)

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

[[[ 0.9897232  -0.02159078 -0.32395105 ... -0.44993981  0.78792312
    0.15481593]
  [ 0.95664717 -1.24601022  1.11721664 ... -0.03676713  0.23609711
    0.29997597]
  [ 0.40455987 -0.08662261 -0.01812397 ...  0.83525602  0.14225209
    0.15171397]
  ...
  [ 0.25236402  0.11653272 -0.07920529 ...  0.11802031  0.13952081
    0.40246946]
  [ 0.13753025  0.05674584 -0.00388266 ...  0.1676928   0.27063033
    0.37127483]
  [ 0.0671279  -0.06949123 -0.11600647 ...  0.16091315  0.25414744
    0.3710355 ]]]


In [ ]:
for row in auth3_df.iterrows():
  text = row[1]['Paraphrased Text']
  ls = []
  ls.append(text)
  outputs = class_model.predict(ls)
  last_hidden_state = outputs[3][11]
  matrix = np.add(matrix, last_hidden_state)

datapoints = len(auth3_df)
avg_hidden_layer3 = np.divide(matrix, datapoints)
print(avg_hidden_layer3)

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

[[[ 1.33602437 -0.02044361 -0.4281312  ... -0.64496478  1.18153618
    0.24451979]
  [ 1.33775638 -1.70741225  1.54586642 ... -0.24643699  0.39905022
    0.44554305]
  [ 0.69274668 -0.263237   -0.08248152 ...  0.89243622  0.31240496
    0.40110038]
  ...
  [ 0.31907544  0.05238458 -0.022779   ...  0.13414942  0.22977325
    0.49558135]
  [ 0.20463176 -0.0221672   0.05138299 ...  0.21308161  0.40497987
    0.49884386]
  [ 0.10062522 -0.12758132 -0.1083588  ...  0.18106531  0.400345
    0.48951562]]]


In [ ]:
!pip install scipy
!pip install sklearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for sklearn: filename=sklearn-0.0.post4-py3-none-any.whl size=2973 sha256=a3b67ef8a3ec62141c6afc43f87e70f20ac393e9972f9b9738b11097cf5418ca
  Stored in directory: /root/.cache/pip/wheels/46/b8/69/4f7789b7d2fc0718a44cbc4ae5d40970a40c22c472ae50f0a1
Successfully built sklearn


In [ ]:
from scipy.spatial import distance
def mahalanobis(P, Q):
    stacked_matrix = np.stack((P, Q), axis=0)
    covariance_matrix = np.cov(stacked_matrix)
    inverse_covariance_matrix = np.linalg.inv(covariance_matrix)
    return distance.mahalanobis(P, Q, inverse_covariance_matrix)

def bhattacharyya_distance(P, Q):
    STEPS = 100
    dP = gaussian_kde(P)
    cov_factor = 0.1
    dP.covariance_factor = lambda:cov_factor
    dP._compute_covariance()

    dQ = gaussian_kde(Q)
    cov_factor = 0.1
    dQ.covariance_factor = lambda:cov_factor
    dQ._compute_covariance()

    combinedProb = np.concatenate((P, Q))

    coef = np.linspace(np.min(combinedProb), np.max(combinedProb), STEPS)
    bhatt_coef = 0
    for c in coef:
        p = dP(c)
        q = dQ(c)
        bhatt_coef += np.sqrt(p*q) * (np.min(combinedProb), np.max(combinedProb)) / STEPS

    if bhatt_coef == 0:
        return float('Inf')
    
    return -np.log(bhatt_coef)

In [ ]:
from sklearn.preprocessing import normalize
from scipy.special import expit
p0 = np.squeeze(avg_hidden_layer)
p1 = np.squeeze(avg_hidden_layer1)
p2 = np.squeeze(avg_hidden_layer2)
p3 = np.squeeze(avg_hidden_layer3)
normp0 = expit(p0)
normp1 = expit(p1)
normp2 = expit(p2) #normalize(p2, axis=0, norm='l1')
normp3 = expit(p3) # normalize(p3, axis=0, norm='l1')
dist0_1 = distance.jensenshannon(normp0, normp1)
dist0_2 = distance.jensenshannon(normp0, normp2)
dist0_3 = distance.jensenshannon(normp0, normp3)

dist1_2 = distance.jensenshannon(normp1, normp2)
dist1_3 = distance.jensenshannon(normp1, normp3)

dist2_3 = distance.jensenshannon(normp2, normp3)


In [ ]:
print(np.average(dist1_3))
print(np.average(dist1_2))

0.027197188632284944
0.01575941880858366


In [62]:
print(np.average(dist0_1))
print(np.average(dist0_2))
print(np.average(dist0_3))
print(np.average(dist1_2))
print(np.average(dist1_3))
print(np.average(dist2_3))

0.016743555798153842
0.02718817085464545
0.03840446989991839
0.01575941880858366
0.027197188632284944
0.01658472986784213


In [ ]:
avg_hidden_layer2

array([[[ 0.9897232 , -0.02159078, -0.32395105, ..., -0.44993981,
          0.78792312,  0.15481593],
        [ 0.95664717, -1.24601022,  1.11721664, ..., -0.03676713,
          0.23609711,  0.29997597],
        [ 0.40455987, -0.08662261, -0.01812397, ...,  0.83525602,
          0.14225209,  0.15171397],
        ...,
        [ 0.25236402,  0.11653272, -0.07920529, ...,  0.11802031,
          0.13952081,  0.40246946],
        [ 0.13753025,  0.05674584, -0.00388266, ...,  0.1676928 ,
          0.27063033,  0.37127483],
        [ 0.0671279 , -0.06949123, -0.11600647, ...,  0.16091315,
          0.25414744,  0.3710355 ]]])